In [1]:
import pandas as pd

data = pd.read_excel("Återbet.xlsx")

data = data[pd.to_numeric(data['OrderNo'], errors='coerce').notnull()]

data["OrderNo"] = data["OrderNo"].astype(int).replace('\.o', '', regex = True)

data["PriceIncVat"] = data["PriceIncVat"].astype(int).replace('\.o', '', regex = True)

data = data[["ProductName", "OrderNo", "PriceIncVat", "Info"]]

data.dropna(subset=['ProductName'], inplace = True)

data["Konto"] = 1570

data["PriceIncVat"] = data["PriceIncVat"]*-1

data

# Det man skulle vilja göra är att lägga på 19 kr på ordrar där det är hemleverans
# samt sedan summera allt man lagt på och bokföra det som frakt (konto 3507/920 och 2610)

,ProductName,OrderNo,PriceIncVat,Info,Konto
0,Paketentré web Sommar,591909,-280,", Klarna, Hemleverans",1570
1,Rab-Åkpaketet låg web,591909,100,", Klarna, Hemleverans",1570
2,Åkband web paket,591909,-710,", Klarna, Hemleverans",1570
3,Bonusåkband,591909,0,", Klarna, Hämta i park",1570
4,Guldkortet ÅA,592470,-1599,", Klarna, Digitalt",1570
...,...,...,...,...,...
107,Guldkortet ÅA,600198,-1599,", Klarna, Digitalt",1570
108,Guldkortet ÅA,600549,-1599,", Klarna, Digitalt",1570
109,Guldkortet ÅA,600549,-1599,", Klarna, Digitalt",1570
110,Guldkortet web,600638,-1699,", Klarna, Hämta i park",1570


In [2]:
t = data[data.Info.str.contains('Hemleverans')].drop_duplicates(subset='OrderNo')
t = t['PriceIncVat'].apply(lambda x: x-19)

t

0      -299
7      -149
9      -269
16     -329
21     -279
24     -409
31     -149
34    -1718
36     -329
43    -1718
46    -1718
49    -1718
50     -539
54     -279
58    -1718
68    -1718
76    -1718
79    -1718
82     -329
87    -1718
104   -1718
111    -519
Name: PriceIncVat, dtype: int64

In [3]:
t.count()

22

In [4]:
def frakt(data):
    try:
        antal = t.count()
        split = [
        {'Konto':3507, 'B': 920, 'Belopp': 15.2*antal, 'Referens': ''},
        {'Konto':2610, 'B': None, 'Belopp': 3.8*antal, 'Referens': ''},
        ] 
        split = pd.DataFrame(split)
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

In [5]:
data6 = frakt(data)

data6

,Konto,B,C,D,E,Belopp,Referens
0,3507,920.0,None,None,None,334.4,
1,2610,NaN,None,None,None,83.6,


In [6]:
# Replace values by index

my_list= t.index.values.tolist()


In [7]:
data.at[my_list,"PriceIncVat"]= t

data

,ProductName,OrderNo,PriceIncVat,Info,Konto
0,Paketentré web Sommar,591909,-299,", Klarna, Hemleverans",1570
1,Rab-Åkpaketet låg web,591909,100,", Klarna, Hemleverans",1570
2,Åkband web paket,591909,-710,", Klarna, Hemleverans",1570
3,Bonusåkband,591909,0,", Klarna, Hämta i park",1570
4,Guldkortet ÅA,592470,-1599,", Klarna, Digitalt",1570
...,...,...,...,...,...
107,Guldkortet ÅA,600198,-1599,", Klarna, Digitalt",1570
108,Guldkortet ÅA,600549,-1599,", Klarna, Digitalt",1570
109,Guldkortet ÅA,600549,-1599,", Klarna, Digitalt",1570
110,Guldkortet web,600638,-1699,", Klarna, Hämta i park",1570


In [8]:
data["B"] = None
data["C"]= None
data["D"] = None
data["E"] = None

data = data[["Konto", "B", "C", "D", "E", "PriceIncVat", "OrderNo"]]

data.columns = ["Konto", "B", "C", "D", "E","Belopp", "Referens"]

data

,Konto,B,C,D,E,Belopp,Referens
0,1570,None,None,None,None,-299,591909
1,1570,None,None,None,None,100,591909
2,1570,None,None,None,None,-710,591909
3,1570,None,None,None,None,0,591909
4,1570,None,None,None,None,-1599,592470
...,...,...,...,...,...,...,...
107,1570,None,None,None,None,-1599,600198
108,1570,None,None,None,None,-1599,600549
109,1570,None,None,None,None,-1599,600549
110,1570,None,None,None,None,-1699,600638


In [9]:
data6

,Konto,B,C,D,E,Belopp,Referens
0,3507,920.0,None,None,None,334.4,
1,2610,NaN,None,None,None,83.6,
